In [16]:
import pysolr
import spacy
from spacy import displacy

In [6]:
nlp = spacy.load("/veld/input/m3/model/model-best/")

/usr/local/lib/python3.10/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'de_pipeline' (0.0.0) was trained with spaCy v3.6 and may not be 100% compatible with the current version (3.5.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
solr_core_url = "http://linkedcat-solr.acdh-cluster-2.arz.oeaw.ac.at/solr/linkedcat2"
#pagination_limit = 999999
pagination_limit = 3

In [4]:
solr = pysolr.Solr(solr_core_url, always_commit=True)

In [34]:
solr_data = list(solr.search("*:*", rows=pagination_limit))

In [42]:
doc_list = []
for entry in solr_data:
    text = entry["ocrtext_good"][0]
    doc = nlp(text)
    print(entry["id"], len(doc.ents))
    doc_list.append((entry["id"], doc))

AC15207455 94
AC15207473 250
AC15207506 66


In [38]:
def print_with_ner(doc):
    displacy.render(doc, style="ent", jupyter=True)

In [39]:
print_with_ner(solr_data[0])

AttributeError: 'dict' object has no attribute 'ents'

In [33]:
doc = nlp(solr_data[0]["ocrtext_good"][0])
displacy.render(doc, style="ent", jupyter=True)